In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas import Series
from time import sleep
from geopy.geocoders import Nominatim

#importing the dataset using pandas
df = pd.read_csv("./911.csv")

#sample of original dataset
df.head(5)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1


In [2]:
#separate timeStamp data into 2 new columns
df['date'] = df.timeStamp.str[0:11]
df['time'] = df.timeStamp.str[-8:]

#Get rid of dummy 'e' column and 'timeStamp' column
del df['e']
del df['timeStamp']
#If time at end, then try to extract station number and impute
del df['desc']

#sample of dataset
#df.head(10)

In [3]:
#count number of values in each column
df.count()

lat      177755
lng      177755
zip      155957
title    177755
twp      177694
addr     177755
date     177755
time     177755
dtype: int64

In [4]:
#Finds total number of rows with missing values 
#Rows with more than one missing value only counted once
df.isnull().any(axis=1).sum()

21822

In [ ]:
#Drop rows with missing zipcode values
df = df.dropna(subset=['zip'])

#Convert float values for zipcodes to integer type
df['zip'] = df['zip'].astype(int)
df.count()

lat      155957
lng      155957
zip      155957
title    155957
twp      155933
addr     155957
date     155957
time     155957
dtype: int64

In [ ]:
empty = np.where(pd.isnull(df))
geolocator = Nominatim()
index = 0


#Impute 24 missing township values
for i in np.nditer(empty):
    
    #row of missing township cell
    row = empty[0][index] 
    #column of missing township cell
    column = empty[1][index]
    
    
    temp_lat = repr(df.iloc[row,0])
    temp_long = repr(df.iloc[row,1])
    
    
    location = geolocator.reverse([temp_lat, temp_long])
    
    #extract township value from location dictionary
    town = location.raw['address']['city'] 
    
    
    #remove 'Township' ending from name of town    
    if town.endswith("Township"):
        town = town[0:-9]
            
    else:
        pass
                
    #convert to uppercase to maintain township format in dataframe    
    town = town.upper()
    
    #put imputed township name into corresponding missing cell of dataframe
    df.iloc[row, column] = town
    
    print(df.iloc[row, column])
             
    
    #required waiting period to prevent timeout exception from being thrown
    #this requriement would cause imputing the 21,798 missing zip codes to take approx. 12-20 hours to impute
    sleep(2)
    
        
    #increment index to get to next set of index values for empty township cell
    index += 1

UPPER MORELAND
WARRINGTON
NORRISTOWN
NORRISTOWN


In [ ]:
df.count()

In [ ]:
hour = df.time.str[0:2]
hour2 = pd.to_numeric(hour)

#If time of call is between 6PM and 6AM then it is classified as 'night', otherwise it is classified as 'day'

for i, row in df.iterrows():
    if(hour2.loc[i] >= 18 or hour2.loc[i] < 6):
        hour.at[i] = 'night'
    else:
        hour.at[i] = 'day'


In [ ]:
#Replace military time with either 'night' or 'day'
del df['time']

df['time_of_day'] = hour

#df.head(10)   

In [ ]:
#Change date format to weekdays format
df['dates'] = pd.to_datetime(df['date'])
df['weekday'] = df['dates'].dt.weekday_name

del df['date']
del df['dates']

In [ ]:
#Separate first part of 911 call classification from rest of title
df['class'], df['title2'] = df['title'].str.split(':', 1).str
del df['title']
del df['title2']


In [ ]:
df.head(10)

In [ ]:
#unique zipcodes in the dataset
df.zip.unique()

In [ ]:
#find the number of unique zipcodes
s = Series(df.zip)
zip_unique = s.unique().size
print(zip_unique)

In [ ]:
#unique townships in the dataset
df.twp.unique()

In [ ]:
#find the number of unique townships
s2 = Series(df.twp)
twp_unique = s2.unique().size
print(twp_unique)

In [ ]:
#find the number of unique address locations
s4 = Series(df.addr)
s4.unique().size

In [ ]:
sLat = Series(df.lat)
sLat.unique().size

In [ ]:
sLong = Series(df.lng)
sLong.unique().size

In [ ]:
sLat.max()

In [ ]:
sLat.min()

In [ ]:
sLong.max()

In [ ]:
sLong.min()

In [ ]:
df.weekday.shape
from sklearnmodel.selection import train_test_split
X, y = df['twp'], df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .8, random_state = 42)
target_class = ['EMS', 'Fire', 'Traffic']